In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from split import acquire_train_test_data
import warnings
warnings.filterwarnings("ignore")

In [4]:
train,test=acquire_train_test_data()
train,validate=train_test_split(train, train_size=.8, random_state=42, stratify = train.hospital_death)

# Build grid search function

In [6]:
y_train = train['hospital_death']
X_train = train.drop(columns='hospital_death')
y_validate = validate['hospital_death']
X_validate = validate.drop(columns='hospital_death')

In [19]:
dt = DecisionTreeClassifier(max_depth=10)
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [16]:
dt_probs_tra = dt.predict_proba(X_train)
dt_probs_val = dt.predict_proba(X_validate)
dt_probs_tra = dt_probs_tra[:, 1]
dt_probs_val = dt_probs_val[:, 1]
score_tra= roc_auc_score(y_train, dt_probs_tra)
score_val= roc_auc_score(y_validate, dt_probs_val)
score = (score_tra, score_val)

In [17]:
score

(0.8899811118998381, 0.8224021899165621)

In [22]:
max_depth = list(range(1,5))
max_depth

[1, 2, 3, 4]

In [ ]:

def grid_search(estimator,params):
    for depth in max_depth:
        dt = DecisionTreeClassifier(max_depth=depth)
        dt.fit(X_train, y_train) 
        dt_probs_tra = dt.predict_proba(X_train)
        dt_probs_val = dt.predict_proba(X_validate)
        dt_probs_tra = dt_probs_tra[:, 1]
        dt_probs_val = dt_probs_val[:, 1]
        score_tra= roc_auc_score(y_train, dt_probs_tra)
        score_val= roc_auc_score(y_validate, dt_probs_val)
        score = (score_tra, score_val)
    

In [28]:
for key in params:
    for n in params[key]:
        print(key, '=', n)

max_depth = 1
max_depth = 2
max_depth = 3
max_depth = 4
criterion = gini
criterion = entropy


In [ ]:
new_list = [something(key, value) for key, value in a_dict.items()]

In [25]:
params = {
    'max_depth': list(range(1, 5)),
    'criterion': ['gini', 'entropy']
}

In [ ]:
# cv=4 means 4-fold cross-validation, i.e. k = 4
grid = GridSearchCV(dtree, params, cv=4,scoring=roc_auc_scorer)
grid.fit(X, y)

grid.best_params_